In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

In [2]:
df_drl = pd.read_excel("./trading_records/dax_trading_records.xlsx")

In [3]:
df_drl

Unnamed: 0   ticker       date  signal  cash_balance  share_holding  \
0               0   ADS.DE 2018-11-01       0  1.000000e+06              0   
1               1   ADS.DE 2018-11-02       1  7.545209e+01           5015   
2               2   ADS.DE 2018-11-05      -1  9.955792e+05              0   
3               3   ADS.DE 2018-11-06       0  9.955792e+05              0   
4               4   ADS.DE 2018-11-07      -1  9.955792e+05              0   
...           ...      ...        ...     ...           ...            ...   
19651         751  VOW3.DE 2021-10-25      -1  1.237850e+06              0   
19652         752  VOW3.DE 2021-10-26       1  1.262965e+02           6032   
19653         753  VOW3.DE 2021-10-27       0  1.262965e+02           6032   
19654         754  VOW3.DE 2021-10-28       1  1.262965e+02           6032   
19655         755  VOW3.DE 2021-10-29       1  1.262965e+02           6032   

              asset  transaction_price  transaction_cost  trading_outlay  \
0      1.000000e+06         197.048916          0.000000    0.000000e+00   
1      9.942750e+05         198.245183        600.594372    9.999245e+05   
2      9.955792e+05         197.958070        598.661441    0.000000e+00   
3      9.955792e+05         197.096758          0.000000    0.000000e+00   
4      9.955792e+05         190.660855          0.000000    0.000000e+00   
...             ...                ...               ...             ...   
19651  1.237850e+06         200.644997        744.131743    0.000000e+00   
19652  1.241814e+06         205.850006        743.188287    1.237724e+06   
19653  1.227488e+06         203.475006          0.000000    1.237724e+06   
19654  1.190180e+06         197.289993          0.000000    1.237724e+06   
19655  1.169791e+06         193.910004          0.000000    1.237724e+06   

            reward  
0         0.000000  
1         0.000000  
2     -4420.807106  
3         0.000000  
4         0.000000  
...            ...  
19651  9582.791242  
19652     0.000000  
19653     0.000000  
19654     0.000000  
19655     0.000000  

[19656 rows x 11 columns]

In [4]:
df_etf = yf.Ticker("EXS1.DE").history(start="2018-11-01", end="2021-11-01")

In [5]:
def get_returns(prices):
    return prices[-1]/prices[0]-1

In [14]:
df_sum = pd.DataFrame(df_drl.groupby(["date"]).asset.apply(np.sum))
monthly_return_drl = df_sum.groupby([df_sum.index.year, df_sum.index.month]).asset.apply(get_returns)
monthly_return_etf = df_etf.groupby([df_etf.index.year, df_etf.index.month]).Close.apply(get_returns)

In [15]:
monthly_return_drl = np.resize([np.nan]*10 + list(monthly_return_drl.values) + [np.nan]*2, (4, 12))
monthly_return_etf = np.resize([np.nan]*10 + list(monthly_return_etf.values) + [np.nan]*2, (4, 12))

In [16]:
import plotly.graph_objects as go

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = [i for i in np.arange(2018, 2021+1, 1)]
monthly_returns = monthly_return_drl

annotations = go.Annotations()
for n, row in enumerate(monthly_returns):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text=str(round(monthly_returns[n][m]*100,2)), x=months[m], y=years[n],
                                         xref='x1', yref='y1', showarrow=False))
colorscale = [[0, '#ef476f'], [0.5, 'white'], [1, '#06d6a0']]

trace = go.Heatmap(x=months, y=years, z=monthly_returns, colorscale=colorscale, showscale=True)

fig = go.Figure(data=go.Data([trace]))
fig['layout'].update(
    title="Monthly returns in a heat map (%)",
    annotations=annotations,
    xaxis=go.XAxis(ticks='', side='top'),
    yaxis=go.YAxis(ticks='', ticksuffix='  '),  # ticksuffix is a workaround to add a bit of padding
#     width=1200,
#     height=700,
    autosize=True
)

fig.show()

/home/zj/anaconda3/envs/rllib_test/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:68: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/home/zj/anaconda3/envs/rllib_test/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:149: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/home/zj/anaconda3/envs/rllib_test/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogra

In [17]:
import plotly.graph_objects as go

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = [i for i in np.arange(2018, 2021+1, 1)]
monthly_returns = monthly_return_etf

annotations = go.Annotations()
for n, row in enumerate(monthly_returns):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text=str(round(monthly_returns[n][m]*100,2)), x=months[m], y=years[n],
                                         xref='x1', yref='y1', showarrow=False))
colorscale = [[0, '#ef476f'], [0.5, 'white'], [1, '#06d6a0']]

trace = go.Heatmap(x=months, y=years, z=monthly_returns, colorscale=colorscale, showscale=True)

fig = go.Figure(data=go.Data([trace]))
fig['layout'].update(
    title="Monthly returns in a heat map (%)",
    annotations=annotations,
    xaxis=go.XAxis(ticks='', side='top'),
    yaxis=go.YAxis(ticks='', ticksuffix='  '),  # ticksuffix is a workaround to add a bit of padding
#     width=1200,
#     height=700,
    autosize=True
)

fig.show()

In [18]:
import plotly.graph_objects as go

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = [i for i in np.arange(2018, 2021+1, 1)]
monthly_returns = monthly_return_drl - monthly_return_etf

annotations = go.Annotations()
for n, row in enumerate(monthly_returns):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text=str(round(monthly_returns[n][m]*100,2)), x=months[m], y=years[n],
                                         xref='x1', yref='y1', showarrow=False))
colorscale = [[0, '#ef476f'], [0.5, 'white'], [1, '#06d6a0']]

trace = go.Heatmap(x=months, y=years, z=monthly_returns, colorscale=colorscale, showscale=True)

fig = go.Figure(data=go.Data([trace]))
fig['layout'].update(
    title="Monthly returns in a heat map (%)",
    annotations=annotations,
    xaxis=go.XAxis(ticks='', side='top'),
    yaxis=go.YAxis(ticks='', ticksuffix='  '),  # ticksuffix is a workaround to add a bit of padding
#     width=1200,
#     height=700,
    autosize=True
)

fig.show()

/home/zj/anaconda3/envs/rllib_test/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:68: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/home/zj/anaconda3/envs/rllib_test/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:149: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/home/zj/anaconda3/envs/rllib_test/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogra

In [19]:
yearly_return_drl = df_sum.groupby([df_sum.index.year]).asset.apply(get_returns)
yearly_return_etf = df_etf.groupby([df_etf.index.year]).Close.apply(get_returns)

In [22]:
# define histogram colors
colors_macd = np.where(yearly_return_drl.values > 0, '#06d6a0', '#ef476f')
colors_etf = np.where(yearly_return_etf.values > 0, '#0a9396', '#e63946')

# add histogram on the lower subplot
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=yearly_return_drl.index,
        y=yearly_return_drl.values * 100,
        name="DRL",
        marker_color=colors_macd,
        text=yearly_return_drl.values * 100,
        texttemplate='%{text:.2f}',
        textposition='outside'

    ),
)
fig.add_trace(
    go.Bar(
        x=yearly_return_etf.index,
        y=yearly_return_etf.values * 100,
        name='ETF',
        marker_color=colors_etf,
        text=yearly_return_etf.values * 100,
        texttemplate='%{text:.2f}',
        textposition='outside'

    ),
)

# make the figure prettier
layout = go.Layout(
    title="Yearly asset return comparison with ETF (%)",
    plot_bgcolor='#ecf8f8',
    font_family='Monospace',
    font_color='#073b4c',
    font_size=10,
    xaxis=dict(
        rangeslider=dict(visible=False)
    ),
    autosize=True,
)

fig.update_layout(layout)
fig.show()

In [36]:
import datetime
import dateutil.relativedelta

end_period = datetime.datetime.strptime('2021-10-31', "%Y-%m-%d")

In [37]:
end_close_etf = df_etf[df_etf.index <= end_period].iloc[-1].Close

In [38]:
periods = [1, 3, 6, 1*12, 3*12]
period_return_etf = []
for i in periods:
    period_return_etf.append(end_close_etf/df_etf[df_etf.index > end_period - dateutil.relativedelta.relativedelta(months=i)].iloc[0].Close-1)

In [39]:
end_close_drl = df_sum[df_sum.index <= end_period].iloc[-1].asset

In [43]:
periods = [1, 3, 6, 1*12, 3*12]
period_return_drl = []
for i in periods:
    period_return_drl.append(end_close_drl/df_sum[df_sum.index > end_period - dateutil.relativedelta.relativedelta(months=i)].iloc[0].asset-1)

In [48]:
# define histogram colors
colors_drl = np.where(np.array(period_return_drl) > 0, '#06d6a0', '#ef476f')
colors_etf = np.where(np.array(period_return_etf) > 0, '#0a9396', '#e63946')

# add histogram on the lower subplot
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=['1M', '3M', '6M', '1Y', '3Y'],
        y=np.array(period_return_drl)*100,
        name="DRL",
        marker_color=colors_drl,
        text=np.array(period_return_drl)*100,
        texttemplate='%{text:.2f}',
        textposition='outside'

    ),
)
fig.add_trace(
    go.Bar(
        x=['1M', '3M', '6M', '1Y', '3Y'],
        y=np.array(period_return_etf)*100,
        name='ETF',
        marker_color=colors_etf,
        text=np.array(period_return_etf)*100,
        texttemplate='%{text:.2f}',
        textposition='outside'

    ),
)

# make the figure prettier
layout = go.Layout(
    title="Yearly asset return comparison with ETF (%)",
    plot_bgcolor='#ecf8f8',
    font_family='Monospace',
    font_color='#073b4c',
    font_size=10,
    xaxis=dict(
        rangeslider=dict(visible=False)
    ),
    autosize=True,
)

fig.update_layout(layout)
fig.show()